In [1]:
# Import necessary libraries
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Input
from keras.layers.noise import AlphaDropout
from keras.layers import Layer
from tensorflow.python.keras import backend as K
from sklearn.model_selection import train_test_split

# Import custom modules
from network import *
from data import *

/Users/dweepa/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Create the base network
network = dict(n_dense=5, dense_units=16, activation='selu', dropout=AlphaDropout, dropout_rate=0.1,
               kernel_initializer='lecun_normal', optimizer='sgd', num_classes=2)

shared_model = create_base_network(**network)

In [3]:
# Create the siamese network
model = create_siamese_network(shared_model)
print("Siamese network model summary")
model.summary()

Siamese network model summary
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 978)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 978)          0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 16)           16752       input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
man_dist_1 (ManDist)            (None, 1)            0           sequential_1[1

In [4]:
import pickle
dbfile = open('../../Data/X_train', 'rb')      
X_train = pickle.load(dbfile)
dbfile = open('../../Data/y_train', 'rb')      
y_train = pickle.load(dbfile)
dbfile = open('../../Data/X_test', 'rb')      
X_test = pickle.load(dbfile)
dbfile = open('../../Data/y_test', 'rb')      
y_test = pickle.load(dbfile)

In [5]:
model.fit([X_train[0],X_train[1]],y_train, verbose=0,epochs=30)

# Sandbox

In [6]:
X_train[0][1][0:10],X_train[1][1][0:10]

(array([-1.0255878 , -0.5268659 ,  0.93751162, -0.21547468,  0.56213748,
         0.19349912,  0.4320128 ,  0.19859782,  0.49415439,  1.27129149]),
 array([10.        ,  7.40159988, -2.45020008,  5.91870022,  7.57660007,
        -6.60559988, -3.51869988,  5.83519983, -6.07600021, -1.39619994]))

In [7]:
y_train[1]

0.0

In [8]:
pred = model.predict([X_train[0],X_train[1]])

In [9]:
pred.shape=(pred.shape[0],)

In [10]:
pred

array([0.0608592 , 1.2646329 , 0.08790207, ..., 0.12847674, 0.09703755,
       0.03837955], dtype=float32)

In [11]:
min(pred)

0.005431652

In [12]:
max(pred)

137.4568

In [13]:
a = [-2.9628181, -2.9369824, -2.8793402, -2.917018 , -2.8999379,
         -2.934715 , -2.854669 , -2.8483472, -2.8471825, -2.8114026,
         -2.8795078, -2.9513776, -2.8242412, -2.8139703, -2.838598 ,
         -2.837599 ]
sum([i*i for i in a])

132.50516110874779

In [14]:
from keras.models import Model

for i in range(0,len(shared_model.layers)):
    intermediate_layer_model = Model(inputs=shared_model.get_input_at(0),
                                     outputs=[shared_model.layers[i].get_output_at(0)])

    intermediate_output = intermediate_layer_model.predict(X_train[0][11:15])
    print(i,"\n",intermediate_output[0],'\n',intermediate_output[1],'\n\n')

0 
 [ 0.9485297  -0.33970293  0.1092995   0.4486537  -1.4878124   0.67521477
  1.5647703  -1.0226723  -1.398706   -0.15589765 -1.6071329  -1.8077804
 -0.18090928 -3.779647    0.01647523  3.7859588 ] 
 [ 1.3704066  -4.3527875   3.5976717  -0.7790788  -5.7916174   0.7740673
 -2.3052375  -2.764337   -1.6100308   4.279731   -2.895475   -3.5928235
 -0.75792587 -7.287158   -4.465814   11.064843  ] 


1 
 [ 0.99662113 -0.50636464  0.11484109  0.4714009  -1.3610041   0.7094489
  1.6441058  -1.1258295  -1.323996   -0.2537874  -1.405668   -1.4697398
 -0.29094604 -1.7179607   0.01731054  3.9779108 ] 
 [ 1.4398876  -1.735471    3.7800775  -0.95143324 -1.7527317   0.8133133
 -1.5827551  -1.6473078  -1.4066879   4.4967175  -1.6609243  -1.7097155
 -0.9341881  -1.7568963  -1.7378893  11.625842  ] 


2 
 [ 0.99662113 -0.50636464  0.11484109  0.4714009  -1.3610041   0.7094489
  1.6441058  -1.1258295  -1.323996   -0.2537874  -1.405668   -1.4697398
 -0.29094604 -1.7179607   0.01731054  3.9779108 ] 
 [ 1.4

In [15]:
shared_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                15664     
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
_________________________________________________________________
alpha_dropout_1 (AlphaDropou (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_2 (Activation)    (None, 16)                0         
_________________________________________________________________
alpha_dropout_2 (AlphaDropou (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
__________

In [30]:
for i in range(0,len(shared_model.layers),3):
    print(shared_model.layers[i].get_weights(),'\n')

[array([[ 0.0457773 ,  0.03268516,  0.00758917, ..., -0.02492453,
        -0.01496286, -0.04287829],
       [-0.00744581, -0.08316295,  0.03640645, ...,  0.01817912,
        -0.09407642,  0.04407138],
       [-0.06289925,  0.03642048, -0.03180946, ..., -0.02706812,
         0.06249278, -0.07062154],
       ...,
       [-0.00594265, -0.06770797,  0.05274   , ...,  0.03303561,
         0.04171108, -0.04106199],
       [ 0.08317158, -0.00234277, -0.02263706, ..., -0.06206778,
        -0.00569473,  0.03929878],
       [ 0.04302384, -0.01728372,  0.03767493, ...,  0.02610114,
        -0.02861562,  0.02661343]], dtype=float32), array([ 0.28650334, -0.3713573 ,  0.27908677, -0.23875605, -0.37324145,
        0.27465007, -0.02009616, -0.02453259, -0.19645755,  0.25295946,
       -0.31927878, -0.3303229 , -0.17784697,  0.25048032, -0.36453685,
        0.19967358], dtype=float32)] 

[array([[ 0.14690769,  0.30641556, -0.15497342, -0.19234574,  0.16280282,
        -0.23269647, -0.10208861,  0.4656